In [ ]:
%pip install huggingface_hub autotrain-advanced

In [ ]:
%autotrain setup --update-torch

> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


In [3]:
from huggingface_hub import notebook_login
import os
import torch
from datasets import load_dataset
import csv
import pandas as pd
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    TextDataset
)
notebook_login()

In [4]:
MODEL_NAME = "TinyPixel/Llama-2-7B-bf16-sharded"
OUTPUT_DIR = "./fine_tuned_"+MODEL_NAME
TOKENIZER_NAME = "TinyPixel/Llama-2-7B-bf16-sharded"
train_file_path = "dataset/dataset.csv"
train_directory_path = "dataset"
dataset_path = "augmented_darwin_dataset.txt"
overwrite_output_dir = False
train_batch_size = "2"
num_train_epochs = "5"
save_steps = "500"
project_name = "llamachatbot"
learning_rate = "2e-4"
model_max_length = "2048"
REPO_ID = "DarwinDontCare/llama_chat_bot_finetune"

In [6]:
#prepare dataset
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)
tokenizer.pad_token = "<[PAD]>"
tokenizer.padding_side = "right"

dataset = open(dataset_path, 'r', encoding="utf-8")
samples = []
for idx, item in enumerate(dataset.readlines()):
    item = item.replace("<|endoftext|><|endoftext|>", "<|endoftext|>")
    item = item.replace("<|endoftext|>", tokenizer.eos_token)
    item = item.replace("  ", " ")
    item = item.replace("<[USER]> :", "### User: ")
    item = item.replace("<[SYSTEM]> :", "### System: ")
    item = item.replace("<[AI]> :", "### AI: ")
    item = tokenizer.bos_token+item
    samples.append({"text": item.strip()})

with open(train_file_path, "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["text"])
    writer.writeheader()
    writer.writerows(samples)

tokenizer.save_pretrained("tokenizer")

('./fine_tuned_TinyPixel/Llama-2-7B-bf16-sharded\\tokenizer_config.json',
 './fine_tuned_TinyPixel/Llama-2-7B-bf16-sharded\\special_tokens_map.json',
 './fine_tuned_TinyPixel/Llama-2-7B-bf16-sharded\\tokenizer.model',
 './fine_tuned_TinyPixel/Llama-2-7B-bf16-sharded\\added_tokens.json',
 './fine_tuned_TinyPixel/Llama-2-7B-bf16-sharded\\tokenizer.json')

In [7]:
#train model
command = f"autotrain llm --train --project-name {project_name} --model {MODEL_NAME} --data-path \"{train_directory_path}\" --quantization int4 --lr {learning_rate} --batch-size {train_batch_size} --epochs {num_train_epochs} --trainer sft --block-size {model_max_length} --block-size 2048 > training.log &"

!{command}

> INFO    Running LLM
> INFO    Params: Namespace(version=False, text_column='text', rejected_text_column='rejected', prompt_text_column='prompt', model_ref=None, warmup_ratio=0.1, optimizer='adamw_torch', scheduler='linear', weight_decay=0.0, max_grad_norm=1.0, add_eos_token=False, block_size=2048, peft=False, lora_r=16, lora_alpha=32, lora_dropout=0.05, logging_steps=-1, evaluation_strategy='epoch', save_total_limit=1, save_strategy='epoch', auto_find_batch_size=False, mixed_precision=None, quantization='int4', model_max_length=1024, trainer='sft', target_modules=None, merge_adapter=False, use_flash_attention_2=False, dpo_beta=0.1, chat_template=None, padding=None, train=True, deploy=False, inference=False, username=None, backend='local-cli', token=None, repo_id=None, push_to_hub=False, model='TinyPixel/Llama-2-7B-bf16-sharded', project_name='llamachatbot', seed=42, epochs=5, gradient_accumulation=1, disable_gradient_checkpointing=False, lr=0.0002, log='none', data_path='dataset', tr

In [ ]:
#test model
text_generator = pipeline("text-generation", model=project_name, tokenizer=OUTPUT_DIR)

print("type 'quit' to exit test\n")

context = []

while True:
    text = input("<[USER]> : ")
    if text == "quit":
        break
    else:
        context_text = "null"
        if len(context) > 0:
            context_text = " ".join(context)
        context.append("### User:" +text)
        text = "### User: { \"context\": \""+context_text+"\", \"text\": \""+text+"\", \"isCreator\": \" true \" } ### AI:"
        generated_text = text_generator(text, max_length=500, do_sample=True, top_k=50, top_p=0.95)[0]["generated_text"].replace("\" ", "\"").replace(" \"", "\"")
        response = ""
        if "### AI:" in generated_text:
            response = json.loads(generated_text.split("### AI:")[1])
        if len(response) > 0: 
            if len(response['text']) > 0:
                context.append(response['text'])
                print("<AI> : "+ response['text'])
            if len(response['action']) > 0:
                for action in response['action']:
                    print(f"action: {action['type']}")
                    try:
                        print(f"parameters: {action['data']}")
                    except:
                        pass

OSError: ./fine_tuned_georgesung/llama2_7b_chat_uncensored does not appear to have a file named config.json. Checkout 'https://huggingface.co/./fine_tuned_georgesung/llama2_7b_chat_uncensored/None' for available files.